In [1]:
import json
import pymongo
from pymongo.database import Database
from pymongo.collection import Collection
from time import time
import datetime
with open("watermelon.config", "rb") as f:
    js = json.load(f)
    mongo_key: str = js["mongo_key"]
    prefix: str = js["prefix"]

if prefix in ["w?", "t?"]:  # only access mongodb for w? and t?
    client = pymongo.MongoClient(mongo_key)
    db: Database = client.get_database("AlexMindustry")
    expgains: Collection = db["expgains"]
    convertedexp: Collection = db["convertedexp"]
    muuid: Collection = db["muuid"]
    # V7 stuff
    expv7: Collection = db["expv7"]
    convertedexpv7: Collection = db["convertedexpv7"]
    ingamecosmeticsv7: Collection = db["ingamecosmeticsv7"]
    serverplayerupdates: Collection = db["serverplayerupdates"]
    hexv7: Collection = db["hexdataV7"]

In [2]:
t1=time()
alexid=612861256189083669
xxx1=315764312700485632 #cohl
xxx2=1185756027929501718 #cen
duuid=alexid
t2=time()

## this notebook filters documents with duplicated muuid,usid,ip. and keeps the unique one with the latest date.

In [38]:
%%time
collection = muuid

# Calculate the start date for 2 months ago
for i in range(0,4):
    start_date = datetime.datetime.now() - datetime.timedelta(days=(i+1)*40) #150 to 
    end_date = datetime.datetime.now() - datetime.timedelta(days=i*40)
    
    current_date = start_date
    
    would_delete = []
    
    while current_date < end_date:
        next_date = current_date + datetime.timedelta(days=40)
        
        # Group documents by unique fields excluding _id and date
        pipeline = [
            {
                '$match': {
                    'date': {'$gte': current_date, '$lt': next_date}
                }
            },
            {
                '$group': {
                    '_id': {
                        'musername': '$musername',
                        'muuid': '$muuid',
                        'musid': '$musid',
                        'con_address': '$con_address',
                        'color': '$color',
                        'servername': '$servername'
                    },
                    'latest': {'$max': '$date'},
                    'docs': {'$push': {'_id': '$_id', 'date': '$date'}}
                }
            }
        ]
    
        # Execute the aggregation pipeline
        result = collection.aggregate(pipeline)
    
        # Loop through the aggregation result
        keep=0
        for doc in result:
            latest_date = doc['latest']
            docs = doc['docs']
            latest_id = None
    
            # Find the document with the latest date
            for d in docs:
                if d['date'] == latest_date:
                    latest_id = d['_id']
    
            # Print all other documents except the latest one
            if latest_id:
                keep+=1
                for d in docs:
                    if d['_id'] != latest_id:
                        #print(f"Would delete document: {d['_id']}")
                        would_delete.append(d['_id'])
        print("docs",len(would_delete),"keep",keep,next_date)
    
        current_date = next_date
    
    print("Run complete!")
    res = muuid.delete_many({'_id': {'$in': would_delete}})
    print("delted",res.deleted_count)

docs 690 keep 188641 2024-06-22 17:17:44.687712
Run complete!
delted 690
docs 3004 keep 157589 2024-05-13 17:20:24.273383
Run complete!
delted 3004
docs 1008 keep 158618 2024-04-03 17:22:16.759669
Run complete!
delted 1008
docs 4 keep 24270 2024-02-23 17:23:56.078185
Run complete!
delted 4
CPU times: total: 6.44 s
Wall time: 6min 32s


In [13]:
%%time
from bson.objectid import ObjectId

object_id_list = [ObjectId(idd) for idd in would_delete]
query = {'_id': {'$in': object_id_list}}

matching_documents = muuid.find(query)
counts=0
for doc in matching_documents:
    #print(doc)
    counts+=1

CPU times: total: 46.9 ms
Wall time: 7.6 s


In [14]:
counts

13346

In [ ]:
%%time
collection = muuid

# Calculate the start date for 2 months ago
for i in range(0,4):
    start_date = datetime.datetime.now() - datetime.timedelta(days=(i+1)*40) #150 to 
    end_date = datetime.datetime.now() - datetime.timedelta(days=i*40)
    
    current_date = start_date
    
    would_delete = []
    
    while current_date < end_date:
        next_date = current_date + datetime.timedelta(days=40)
        
        # Group documents by unique fields excluding _id and date
        pipeline = [
            {
                '$match': {
                    'date': {'$gte': current_date, '$lt': next_date}
                }
            },
            {
                '$group': {
                    '_id': {
                        'musername': '$musername',
                        'muuid': '$muuid',
                        'musid': '$musid',
                        'con_address': '$con_address',
                        'color': '$color',
                        'servername': '$servername'
                    },
                    'latest': {'$max': '$date'},
                    'docs': {'$push': {'_id': '$_id', 'date': '$date'}}
                }
            }
        ]
    
        # Execute the aggregation pipeline
        result = collection.aggregate(pipeline)
    
        # Loop through the aggregation result
        keep=0
        for doc in result:
            latest_date = doc['latest']
            docs = doc['docs']
            latest_id = None
    
            # Find the document with the latest date
            for d in docs:
                if d['date'] == latest_date:
                    latest_id = d['_id']
    
            # Print all other documents except the latest one
            if latest_id:
                keep+=1
                for d in docs:
                    if d['_id'] != latest_id:
                        #print(f"Would delete document: {d['_id']}")
                        would_delete.append(d['_id'])
        print("docs",len(would_delete),"keep",keep,next_date)
    
        current_date = next_date
    
    print("Run complete!")
    res = muuid.delete_many({'_id': {'$in': would_delete}})
    print("delted",res.deleted_count)